# [DEV SCRIPT] Feedforward and feedback interactions for sequence learning in the macaque frontotemporal network

In [ ]:
### Import toolboxes and dependencies -----------------------------------------------------#
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import h5py

#------------------------------------------------------------------------------------------#

### Add root & subdirectories to path -----------------------------------------------------#
def init_path(root_dir):
    """
    Recursively add a directory and all its subdirectories to the system path.
    """
    for dirpath, dirnames, filenames in os.walk(root_dir):
        if dirpath not in sys.path:
            sys.path.append(dirpath)
                  
          
if sys.platform == 'win32':  
    init_path(r'C:\KIKUCHI-LOCAL\script\2024-aglt-laminar')
elif sys.platform == 'darwin':
    init_path(r'/Users/stevenerrington/Desktop/Projects/2024-aglt-laminar')
    
    
#------------------------------------------------------------------------------------------#

### Import custom functions ---------------------- ----------------------------------------#
from setup_workspace import *
from matlab_functions import *
from spike_functions import *
#------------------------------------------------------------------------------------------#

### Import experiment logs and define data directories ------------------------------------#
dirs = set_directories()                                    # Get directories
ephysLog, stimulusLog, spike_log = import_exp_map()         # Get experimental logs
ephysLog = clean_exp_map(ephysLog)                          # Clean experiment log
spike_log = clean_spike_map(spike_log)                      # Clean spike log
#------------------------------------------------------------------------------------------#


In [ ]:
session_name = ephysLog['session'][20]
full_path = os.path.join(r'/Users/stevenerrington/Desktop/AGLt_py', session_name + '-events.csv')
event_table = pd.read_csv(full_path)

In [ ]:
session_spikes = spike_log['unitDSP'][spike_log['session'] == session_name]

In [ ]:
for spike_dsp in session_spikes:
    spike_times = np.loadtxt(os.path.join(r'/Users/stevenerrington/Desktop/AGLt_py', session_name + '-' + spike_dsp + '-spk.txt'), delimiter=',')
    spike_times = spike_times.astype(int)

    sdf = spk_convolve(spike_times, np.max(spike_times)+10000, 'Gauss')

    align_times = event_table['stimulusOnset_ms']
    align_times = align_times.to_numpy()
    align_times = align_times.astype(int)
    time_win = np.arange(-1000, 6000);

    sdf_aligned, raster_aligned = spk_align(sdf, spike_times, align_times, time_win)

    mean_data = np.nanmean(sdf_aligned, axis=0)
    
    # Smoothing using a moving average with a window size of 3
    window_size = 200
    window = np.ones(window_size) / window_size
    mean_data_smoothed = np.convolve(mean_data, window, mode='same')
        
    # Plot the mean
    plt.figure(figsize=(10, 6))
    plt.plot(time_win, mean_data, linestyle='-', color='k')
    plt.xlabel('Time from stimulus onset (ms)')
    plt.ylabel('Firing rate')
    plt.show()

In [ ]:

spike_dsp = session_spikes.values[1]

spike_times = np.loadtxt(os.path.join(r'/Users/stevenerrington/Desktop/AGLt_py', session_name + '-' + spike_dsp + '-spk.txt'), delimiter=',')
spike_times = spike_times.astype(int)

sdf = spk_convolve(spike_times, np.max(spike_times)+10000, 'Gauss')

align_times = event_table['stimulusOnset_ms']
align_times = align_times.to_numpy()
align_times = align_times.astype(int)
time_win = np.arange(-1000, 6000);

sdf_aligned, raster_aligned = spk_align(sdf, spike_times, align_times, time_win)

mean_data = np.nanmean(sdf_aligned, axis=0)
# Plot the mean
plt.figure(figsize=(10, 6))
plt.plot(time_win, mean_data, linestyle='-', color='k')
plt.xlabel('Time from stimulus onset (ms)')
plt.ylabel('Firing rate')
plt.show()
